In [20]:
import pandas as pd
import numpy as np

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle, Circle, Ellipse, Patch
from matplotlib.lines import Line2D

from ete3 import Tree

import seaborn as sns
sns.set() # sets seaborn default "prettyness:
sns.set_style("white")
sns.set_context("notebook")

Code for plotting the tree:

In [21]:
def make_comb_tree(t):
    for node in t.traverse():
        if not node.is_leaf():
            node.children = sorted(node.children, key=lambda c: len(c.get_leaves()), reverse=False)

def plot_tree(t, ax, leaf_colors=None, show_inner_nodes=False, flip=False, fontsize=10, 
              text_offset=None, axis=True, linewidth=0.8, markers=True, 
              labels=True, markersize=3, margins=(0.5, 1, 0.5, 1)): # top, right, bottom, left

    y_offset = len(t.get_leaves())
    for node in t.traverse("preorder"):
        node.x_offset = node.dist + sum(x.dist for x in node.get_ancestors())
        if node.is_leaf():
            y_offset -= 1
            node.y_offset = y_offset

    for node in t.traverse("postorder"):
        if not node.is_leaf():
            node.y_offset = sum(x.y_offset for x in node.children) / len(node.children)

    horizontal_lines = list()
    vertical_lines = list()
    node_coords = list()
    leaf_coords = list()
    max_x_offset = 0
    for node in t.traverse("postorder"):
        max_x_offset = max(max_x_offset, node.x_offset)
        node_coords.append((node.x_offset, node.y_offset))
        if node.is_leaf():
            leaf_coords.append([node.name, node.x_offset, node.y_offset])
        if not node.is_root():
            y = node.y_offset
            horizontal_lines.append(([node.up.x_offset, node.x_offset], [y, y]))
        if not node.is_leaf():
            c = sorted(node.children, key=lambda x: x.y_offset)
            bottom, top = c[0], c[-1]
            x = node.x_offset
            vertical_lines.append(([x, x],[bottom.y_offset, top.y_offset]))

    
    # shift the tree to put leaves at zero
    for i in range(len(horizontal_lines)):
        horizontal_lines[i][0][0] -= max_x_offset
        horizontal_lines[i][0][1] -= max_x_offset
    for i in range(len(vertical_lines)):
        vertical_lines[i][0][0] -= max_x_offset
        vertical_lines[i][0][1] -= max_x_offset
    for i in range(len(leaf_coords)):
        leaf_coords[i][1] -= max_x_offset

    if flip:
        l = len(leaf_coords)
        horizontal_lines = [([l-y1-1, l-y2-1], [-x1, -x2]) for ((x1, x2),(y1, y2)) in horizontal_lines]
        vertical_lines = [([l-y1-1, l-y2-1], [-x1, -x2]) for ((x1, x2),(y1, y2)) in vertical_lines]
        leaf_coords = [[n, x, y] for (n, y, x) in leaf_coords]
        
    # draw the tree:
    for x in horizontal_lines:
        ax.plot(*x, c='black', linewidth=linewidth)
    for x in vertical_lines:
        ax.plot(*x, c='black', linewidth=linewidth)

#     for tup in node_coords:
#         ax.plot(*tup, c='black', marker="o")

    if text_offset is None:
        text_offset = max_x_offset / 20

    for name, x, y in leaf_coords:
        if flip:
            verticalalignment='top'
            horizontalalignment='center'
            rotation='vertical'
        else:
            verticalalignment='center'
            horizontalalignment='left'
            rotation='horizontal'
        if labels:
            ax.text(x+text_offset, y, name, fontsize=fontsize,
                    verticalalignment=verticalalignment, 
                    horizontalalignment=horizontalalignment,
                    rotation=rotation)
        if markers:
            if leaf_colors is None:
                color = 'black'
            else:
                color = leaf_colors[name]
            ax.plot(x, y, c=color, marker="o", ms=markersize)

    if flip:
        ax.set_ylim(-margins[1], -(margins[3]-max_x_offset))
        ax.set_xlim(-margins[2], len(leaf_coords)-1+margins[0])
    else:
        ax.set_xlim(-margins[3]-max_x_offset, margins[1])
        ax.set_ylim(-margins[2], len(leaf_coords)-1+margins[0])

    if flip:
        ax.get_xaxis().set_visible(False)
    else:
        ax.get_yaxis().set_visible(False)

    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False)
    if flip:
        ax.yaxis.set_major_locator(plt.MaxNLocator(4))
        ax.spines['bottom'].set_visible(False) 
    else:
        ax.spines['left'].set_visible(False) 
        ax.xaxis.set_major_locator(plt.MaxNLocator(4))
        
    if not axis:
        ax.axis('off')

    
    return leaf_coords            

Read the copying matrix:

In [22]:
# coancestry_matrix = pd.read_csv('Chromopainter/all_autosomes/all_autosomes_mapstate.csv').drop(columns=['relXmat'])
# coincidence_matrix = pd.read_csv('Chromopainter/all_autosomes/all_autosomes_meancoincidence.csv').drop(columns=['relXmat'])
matrix = pd.read_csv('/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/all_autosomes_linked.regionchunkcounts.out',
                    sep=' ', header=0, index_col=0).drop(columns=['num.regions'])
matrix

Sci_16066    Sci_16098     Sci_34449     Sci_34474   Sci_97074  \
Recipient                                                                       
Sci_16066       0.000000  5812.580110    836.850474    803.108950  452.447310   
Sci_16098    5594.710350     0.000000    757.323109    762.639855  407.154902   
Sci_34449    1223.317550  1158.795700      0.000000  12219.615200  646.479155   
Sci_34474    1033.282360   995.999959  10803.033100      0.000000  575.775664   
Sci_97074     329.016088   310.729901    321.719971    322.140227    0.000000   
...                  ...          ...           ...           ...         ...   
PD_0790       837.520926   786.608216   8582.208850  16014.970400  467.208779   
PD_0791       588.036877   554.718707   6503.655920   6307.783300  338.248023   
PD_0792       758.361008   759.058975   8555.293230   7807.769960  442.075027   
PD_0793      1004.486650   953.497768  10191.134600  11198.150500  538.299055   
PD_0794_BAB   805.698274   758.594920  10177.304700   9197.823190  452.814106   

               Sci_97124     PD_0199     PD_0200     PD_0201     PD_0202  ...  \
Recipient                                                                 ...   
Sci_16066     451.938472  422.949525  412.680651  415.659900  417.285959  ...   
Sci_16098     431.309827  380.090627  381.363375  381.448760  393.105849  ...   
Sci_34449     654.004476  584.768421  579.153882  585.278499  578.523408  ...   
Sci_34474     597.502295  515.556419  515.944923  557.605975  531.002922  ...   
Sci_97074    6455.709140  477.831661  502.697919  494.991325  484.096067  ...   
...                  ...         ...         ...         ...         ...  ...   
PD_0790       473.545428  490.901823  501.773745  512.056188  495.034457  ...   
PD_0791       331.590695  357.244832  360.300333  359.239542  352.534471  ...   
PD_0792       446.629323  457.530307  445.213766  481.123573  461.516871  ...   
PD_0793       550.137628  574.186662  579.753679  589.909511  559.790697  ...   
PD_0794_BAB   461.992063  470.455477  466.182752  488.177226  466.193600  ...   

                  PD_0785       PD_0786       PD_0787       PD_0788  \
Recipient                                                             
Sci_16066      685.110957    687.334318    734.210668    663.606388   
Sci_16098      602.006697    612.688091    656.743191    621.277583   
Sci_34449    15292.517100  11361.347400  11603.414000  10483.812500   
Sci_34474     8882.031280  25905.189200  13276.129300   8836.455720   
Sci_97074      284.204378    271.096994    268.694328    280.171163   
...                   ...           ...           ...           ...   
PD_0790       7793.900430  10761.201700  10329.714700   8298.743270   
PD_0791       5274.308080   5807.403330   6818.719650   6372.464910   
PD_0792       8604.893590   8327.580210   8418.802540   7779.610820   
PD_0793      10226.287100   9860.553920  11982.078000  11871.117700   
PD_0794_BAB  22518.659500   7189.898320   8695.798280   8362.473150   

                  PD_0789       PD_0790       PD_0791       PD_0792  \
Recipient                                                             
Sci_16066      683.397967    686.764435    682.015106    661.530894   
Sci_16098      641.158548    615.958762    606.745776    619.666102   
Sci_34449    10506.579200  10646.430000   9951.131570  10423.048600   
Sci_34474    10653.313800  16422.828900   8874.509030   9068.924220   
Sci_97074      272.994929    265.043526    275.626301    279.433292   
...                   ...           ...           ...           ...   
PD_0790      15534.098300      0.000000   8976.496650   9477.019970   
PD_0791       8588.904370   7424.476940      0.000000  14259.540900   
PD_0792      10359.216500   8864.070180  16801.225100      0.000000   
PD_0793      10453.609300  14584.990900  11032.360000   9837.916690   
PD_0794_BAB   8971.204860  11763.758500   8251.863310   7885.834000   

                  PD_0793   PD_0794_BAB  
Recipient        

Read tree from xml (requires lxml package via conda):

In [23]:
df = pd.read_xml('/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/all_autosomes_linked_tree.xml')
tree =  df.Tree[3]

Get order of individuals in tree:

In [24]:
matrix_order = [node.name for node in Tree(tree).traverse("postorder") if node.is_leaf()]

Order matrix accordingly (and reverse x axis to make the plot like Eriks)

In [25]:
ordered_matrix = matrix.loc[matrix_order[::-1], matrix_order]

Somehow get the species color for each individual (these are just dummies):

In [26]:
#df.Pop[2][1:-1].split(')(')

In [27]:
species_colors = dict(zip(matrix_order, 
                       np.random.choice(['#e31a1c', '#FFB100','#33a02c', '#1f78b4'], 
                                        size=len(matrix_order))))
cluster_colors = dict(zip(matrix_order, 
                       np.random.choice(['#e31a1c', '#FFB100','#33a02c', '#1f78b4'], 
                                        size=len(matrix_order))))

In [28]:
idfile_path = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_8_cluster.ids"
idfile = pd.read_csv(idfile_path, sep=" ", names=["PGDP_ID", "pop", "inclusion"])
idfile["ID_index"] = idfile.index

meta_data_samples = pd.read_csv("../data/Papio_metadata_with_clustering.txt", sep =" ")
mapping = {}
for i, row in meta_data_samples.iterrows():
    if row.PGDP_ID[0] != "P":
        mapping["Sci_"+row.PGDP_ID] = row.Origin
    else:
        mapping[row.PGDP_ID] = row.Origin
meta_data_samples["PGDP_ID"] = mapping.keys()
mycols = sns.color_palette(["#BEE39C", "#1F681F", "#FFD062", "#FFE7AF",
                            "#9EADB2", "#258CC1", "#EA3324", "#C06D34"])
name_order = sorted(meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)]["C_origin"].unique())
color_dir = {}
for i in range(len(name_order)):
    color_dir[name_order[i]] = mycols[i]
# This order of color is based on the alphabetical ordering of the 14 clusters
mycols_cmap = ListedColormap(mycols.as_hex())

mycols

[(0.7450980392156863, 0.8901960784313725, 0.611764705882353),
 (0.12156862745098039, 0.40784313725490196, 0.12156862745098039),
 (1.0, 0.8156862745098039, 0.3843137254901961),
 (1.0, 0.9058823529411765, 0.6862745098039216),
 (0.6196078431372549, 0.6784313725490196, 0.6980392156862745),
 (0.1450980392156863, 0.5490196078431373, 0.7568627450980392),
 (0.9176470588235294, 0.2, 0.1411764705882353),
 (0.7529411764705882, 0.42745098039215684, 0.20392156862745098)]

In [29]:
color_df = pd.DataFrame()
color_df["Color"] = meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)].reset_index().C_origin.map(color_dir)
color_df["PGDP_ID"] = meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)].reset_index().PGDP_ID
species_color = pd.Series(color_df["Color"].values,index=color_df["PGDP_ID"]).to_dict()

In [30]:
meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)].Origin.unique()

array(['Mikumi, Tanzania', 'Chunga, Zambia', 'Awash, Ethiopia',
       'Serengeti, Tanzania', 'Arusha, Tanzania', 'Ngorongoro, Tanzania',
       'Ruaha, Tanzania', 'Gombe, Tanzania', 'Udzungwa, Tanzania',
       'Mahale, Tanzania', 'Katavi, Tanzania', 'Selous, Tanzania',
       'Lake Manyara, Tanzania', 'Tarangire, Tanzania',
       'Niokolo-Koba, Senegal', 'Issa Valley, Tanzania',
       'Dendro Park, Zambia', 'Filoha, Ethiopia',
       'Gog Woreda, Gambella region, Ethiopia'], dtype=object)

In [31]:
idfile_path = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_14_cluster.ids"
idfile = pd.read_csv(idfile_path, sep=" ", names=["PGDP_ID", "pop", "inclusion"])
idfile["ID_index"] = idfile.index
idfile_14 = idfile.loc[idfile.PGDP_ID.isin(matrix_order)]

In [32]:
idfile_14.at[189, "pop"] = "cyno_us"
idfile_14.at[119, "pop"] = "cyno_mikumi"

In [33]:
color_14 = sns.color_palette(["#BEE39C", "#88ad00", "#3BAC3B", "#68E068", "#77c458", "#1F681F",
                            "#FFBD00", "#ebc97a", "#FFD062", "#FFDE90",
                            "#9EADB2", "#258CC1", "#EA3324", "#C06D34"])

name_order = ['Anubis, Ethiopia', 'Anubis, Gombe', 'Anubis, Manyara', 'Anubis Ngorongoro and Arusha',
       'Anubis, Serengeti', 'Anubis, Tarangire', 'Cynocephalus, Mikumi',
       'Cynocephalus, Ruaha', 'Cynocephalus, Udzungwa and Selous', 'Cynocephalus, Western Tanzania',
              'Hamadryas', 'Kindae','Papio', 'Ursinus']
l_list = sorted(idfile_14["pop"].unique())
dict_14 = {}
color_dir_14 = {}
for i in range(len(l_list)):
    dict_14[l_list[i]] = name_order[i]
    color_dir_14[name_order[i]] = color_14[i]

In [34]:
idfile_14["name_14"] = idfile_14["pop"].map(dict_14)
idfile_14["color"] = idfile_14["name_14"].map(color_dir_14)
cluster_color = pd.Series(idfile_14["color"].values,index=idfile_14["PGDP_ID"]).to_dict()

/home/eriks/miniconda3/envs/baboondiversity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/eriks/miniconda3/envs/baboondiversity/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
idfile_14.loc[idfile_14.PGDP_ID == "PD_0757"]

PGDP_ID      pop  inclusion  ID_index                            name_14  \
189  PD_0757  cyno_us          1       189  Cynocephalus, Udzungwa and Selous   

                                             color  
189  (1.0, 0.8156862745098039, 0.3843137254901961)

In [36]:
idfile_14.loc[idfile_14["name_14"] == 'Cynocephalus, Udzungwa and Selous']

PGDP_ID      pop  inclusion  ID_index                            name_14  \
34   PD_0223  cyno_us          1        34  Cynocephalus, Udzungwa and Selous   
35   PD_0224  cyno_us          1        35  Cynocephalus, Udzungwa and Selous   
36   PD_0225  cyno_us          1        36  Cynocephalus, Udzungwa and Selous   
37   PD_0226  cyno_us          1        37  Cynocephalus, Udzungwa and Selous   
38   PD_0227  cyno_us          1        38  Cynocephalus, Udzungwa and Selous   
46   PD_0235  cyno_us          1        46  Cynocephalus, Udzungwa and Selous   
47   PD_0236  cyno_us          1        47  Cynocephalus, Udzungwa and Selous   
48   PD_0237  cyno_us          1        48  Cynocephalus, Udzungwa and Selous   
189  PD_0757  cyno_us          1       189  Cynocephalus, Udzungwa and Selous   

                                             color  
34   (1.0, 0.8156862745098039, 0.3843137254901961)  
35   (1.0, 0.8156862745098039, 0.3843137254901961)  
36   (1.0, 0.8156862745098039, 0.3843137254901961)  
37   (1.0, 0.8156862745098039, 0.3843137254901961)  
38   (1.0, 0.8156862745098039, 0.3843137254901961)  
46   (1.0, 0.8156862745098039, 0.3843137254901961)  
47   (1.0, 0.8156862745098039, 0.3843137254901961)  
48   (1.0, 0.8156862745098039, 0.3843137254901961)  
189  (1.0, 0.8156862745098039, 0.3843137254901961)

Plot the figure:

In [37]:
# three panels
fig, ((axA, ax1), (axB, ax2), (axC, ax3), (axD, ax4))  = plt.subplots(4, 2, sharex='col', sharey='row',
                        gridspec_kw={'height_ratios':[10,1,1,40], 
                                     'width_ratios':[1,40],
                                     'hspace':0.01,
                                     'wspace':0.01}, 
                        figsize=(8, 9))

# remove axes we do not need
axA.remove()
axB.remove()
axC.remove()

# plot the tree in top panel
leaf_info = plot_tree(Tree(tree), ax1, flip=True, 
                      axis=False, linewidth=0.6,
                      markers=True, 
                      markersize=1,
                      labels=False,
                      margins=(0, 0, 0, 0), 
                      fontsize=4)

# add colored rectangles for or each individual cluster membership
for i, indiv in enumerate(matrix_order):
    ax2.add_patch(Rectangle((i, 0), 1, 1, linewidth=0, edgecolor='black', facecolor=cluster_color[indiv]))
ax2.text(-1, -0.5, '8 clusters:', fontsize=10, verticalalignment='center', horizontalalignment='right')
ax2.text(-1, 0.5, '14 clusters:', fontsize=10, verticalalignment='center', horizontalalignment='right')  
    
# add colored rectangles for each individual to the middle
#for i, indiv in enumerate(matrix_order):
#    ax3.add_patch(Rectangle((i, 0), 1, 1, linewidth=0, edgecolor='black', facecolor=species_color[indiv]))
for i, indiv in enumerate(matrix_order[::-1]):
    axD.add_patch(Rectangle((0, i), 1, 1, linewidth=0, edgecolor='black', facecolor=species_color[indiv]))
#ax3.text(-30, 0.5, 'Species', fontsize=10, verticalalignment='center', horizontalalignment='right')  

# extra axes for heatmap color bar to allow x sharing of the other three axes
cbar_ax = fig.add_axes([.93, .3, .02, .3])
# plot heatmap in bottom panel
colors = ['white', 'mediumblue', 'darkviolet', 'orangered',  'darkorange'] 
cmap = LinearSegmentedColormap.from_list('custom colormap', colors, N=256)
sns.heatmap(ordered_matrix.to_numpy(), cmap=cmap, ax=ax4, cbar_ax=cbar_ax)
#sns.heatmap(ordered_matrix.to_numpy(), cmap=cmap, ax=ax4, cbar_ax=cbar_ax, vmax=40000)

# add squares with labels along diagonal (maybe to label indentified clusters)
fontsize=11
for label, start, size in [('A', 0, 25), ('B', 25, 26), ('C', 51, 12), ('D', 63, 67),
                          ('E', 130, 27), ('F', 157, 10), ('G', 167, 4), ('H', 171, 52)]:
    ax4.add_patch(Rectangle((start, ordered_matrix.shape[0] - start), size, - size, 
                            linewidth=0.4, edgecolor='black', facecolor='none'))
    ax4.text(start, ordered_matrix.shape[0] - start-size, label, fontsize=fontsize,
                    verticalalignment='bottom', 
                    horizontalalignment='left')    

# some additional squares (maybe to highlight particular admixture signals)
label, x, y, width, height = 'X', 63, 157, 67, 10
ax4.add_patch(Rectangle((x, ordered_matrix.shape[0] - y), width, -height, 
                        linewidth=0.4, edgecolor='black', facecolor='none'))
ax4.text(x, ordered_matrix.shape[0] - y-height, label, fontsize=fontsize,
                verticalalignment='bottom', 
                horizontalalignment='left')    
    
# remove axes from all plots
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
axD.axis('off')

# lines around the colorbar
cbar_ax.axhline(y=0, color='black',linewidth=1)
cbar_ax.tick_params(right=False)

# lines around the matrix
ax4.axhline(y=0, color='black',linewidth=1)
ax4.axvline(x=0, color='black',linewidth=1)
ax4.axhline(y=ordered_matrix.shape[1], color='black',linewidth=1, zorder=1000)
ax4.axvline(x=ordered_matrix.shape[0], color='black',linewidth=1, zorder=1000)

plt.savefig("../figures/fineSTRUCTURE.pdf", dpi=300, bbox_inches='tight')

In [43]:
eastern_yellows = meta_data_samples.loc[meta_data_samples.C_origin == "Cynocephalus, Central Tanzania"].PGDP_ID

In [56]:
meta_data_samples.loc[meta_data_samples.Origin == "Selous, Tanzania"]

PGDP_ID   Provider_ID          Provider  Genus       Species  \
46  PD_0235  01SGM1270217  Knauf/Chuma/Roos  Papio  cynocephalus   
47  PD_0236  04SGM2030317  Knauf/Chuma/Roos  Papio  cynocephalus   
48  PD_0237  07SGF1050317  Knauf/Chuma/Roos  Papio  cynocephalus   

              Origin Sex                                            address  \
46  Selous, Tanzania   M  Selous, Nyangayane Road, Sinza, Dar es Salaam,...   
47  Selous, Tanzania   M  Selous, Nyangayane Road, Sinza, Dar es Salaam,...   
48  Selous, Tanzania   F  Selous, Nyangayane Road, Sinza, Dar es Salaam,...   

    longitude  latitude  callset_index                        C_origin  
46  39.235223  -6.78703             46  Cynocephalus, Central Tanzania  
47  39.235223  -6.78703             47  Cynocephalus, Central Tanzania  
48  39.235223  -6.78703             48  Cynocephalus, Central Tanzania

In [58]:
meta_data_samples.loc[meta_data_samples.PGDP_ID == "PD_0757"]

PGDP_ID Provider_ID Provider  Genus       Species            Origin Sex  \
189  PD_0757  16082_5009  Rogers   Papio  cynocephalus  Mikumi, Tanzania   M   

                                              address  longitude  latitude  \
189  Mikumi, Kilosa, Morogoro, Coastal Zone, Tanzania  36.977032 -7.404852   

     callset_index                        C_origin  
189            189  Cynocephalus, Central Tanzania

In [50]:
ordered_matrix.loc[ordered_matrix.index.isin(eastern_yellows)]

PD_0734     PD_0736     PD_0739     PD_0740     PD_0744  \
Recipient                                                               
PD_0224    500.176518  507.679699  500.688614  495.223576  507.407838   
PD_0227    425.295671  429.847434  445.800570  432.835510  419.976745   
PD_0226    422.782516  424.562972  432.127314  425.440545  430.476426   
PD_0225    434.704754  437.651355  441.833754  439.222325  440.061063   
PD_0223    446.382085  443.219718  453.459305  442.582639  441.939168   
PD_0235    513.196126  513.955443  525.570756  524.536575  518.827709   
PD_0237    415.178678  409.376953  412.868187  417.010047  418.223499   
PD_0236    417.077292  414.640584  412.664906  419.979050  421.497750   
PD_0757    376.167242  382.725732  383.136998  378.076067  386.426302   
PD_0768    310.655007  313.552212  318.285458  319.786622  317.106352   
PD_0767    322.023232  326.017759  332.960584  328.612748  330.296410   
PD_0765    320.228413  316.599782  322.617785  323.439426  321.115387   
PD_0763    315.073271  324.786708  320.527911  324.333087  314.118262   
PD_0754    312.076524  301.745155  307.060159  299.297112  313.240750   
PD_0764    322.423040  316.880237  320.824348  334.203380  322.487413   
PD_0688    314.238386  312.433532  326.490682  320.355714  326.579360   
PD_0270    306.568789  314.324328  310.158640  316.150601  309.992537   
PD_0268    308.509253  316.660070  323.766100  317.850024  318.566202   
PD_0687    338.679069  334.785972  335.408568  330.479431  337.049101   
PD_0271    310.349250  309.043735  313.531271  314.698949  319.573194   
PD_0747    327.315614  328.701565  321.352745  325.549185  335.028699   
PD_0766    351.910877  346.137829  347.941321  348.325022  351.164695   
PD_0762    327.933242  333.306696  332.836010  330.718638  366.278751   
PD_0691    319.643737  328.317991  332.622380  322.074172  324.503955   
Sci_16066  350.751793  351.520294  356.059137  354.150133  360.133858   
PD_0770    296.736663  302.978707  300.366629  302.032563  302.119643   
PD_0774    292.906009  294.991814  294.706080  290.063220  303.052120   
PD_0689    326.815297  328.007562  324.176930  323.922130  323.181374   
PD_0756    317.660983  319.794005  320.576742  326.818095  326.191610   
PD_0690    307.792388  320.244172  313.258718  316.425672  312.851521   
PD_0637    326.577077  332.703332  343.265610  327.072514  325.033974   
PD_0753    310.615230  300.772318  307.361287  310.224522  305.686925   
PD_0748    308.824236  310.717362  302.346547  308.062788  303.312327   
PD_0769    315.029815  316.096039  317.583172  316.305515  314.510492   
PD_0772    314.165597  317.298603  326.488391  323.455770  325.087527   
PD_0773    297.650583  306.312838  306.404616  303.954128  311.224445   
PD_0745    318.740768  323.994572  322.662356  321.534172  321.724719   
Sci_16098  338.821162  350.016626  337.227342  332.178425  329.616997   
PD_0771    331.195249  333.276101  332.694048  328.932866  336.696224   
PD_0269    331.531153  332.563112  337.236188  340.843281  338.622859   
PD_0746    333.854126  367.592773  337.663125  342.106863  337.626411   
PD_0761    347.539525  353.870002  350.683089  356.570591  344.624177   
PD_0272    334.221025  335.899266  338.654638  342.689382  340.190522   
PD_0775    336.060834  340.188245  335.490837  333.975744  342.687565   
PD_0752    349.475137  358.508715  355.206874  358.498640  351.005724   
PD_0755    308.802330  306.882081  315.599918  314.488245  313.082025   
PD_0216    690.724991  715.961943  697.315153  698.108638  698.430690   
PD_0503    665.649166  670.539675  668.495817  680.727599  692.767635   
PD_0217    687.826893  711.823837  722.714379  720.372379  711.280045   
PD_0214    565.710624  577.150506  579.673723  569.814752  571.606558   
PD_0213    550.032000  565.233037  559.347599  564.843852  584.885083   
PD_0215    669.965817  668.619224  665.139406  654.493016  674.641589   

              PD_0721     PD_0726     PD_0722     PD_0720     PD_0743  

In [53]:
ordered_matrix.iloc[:52]

PD_0734     PD_0736     PD_0739     PD_0740     PD_0744  \
Recipient                                                               
PD_0224    500.176518  507.679699  500.688614  495.223576  507.407838   
PD_0227    425.295671  429.847434  445.800570  432.835510  419.976745   
PD_0226    422.782516  424.562972  432.127314  425.440545  430.476426   
PD_0225    434.704754  437.651355  441.833754  439.222325  440.061063   
PD_0223    446.382085  443.219718  453.459305  442.582639  441.939168   
PD_0235    513.196126  513.955443  525.570756  524.536575  518.827709   
PD_0237    415.178678  409.376953  412.868187  417.010047  418.223499   
PD_0236    417.077292  414.640584  412.664906  419.979050  421.497750   
PD_0757    376.167242  382.725732  383.136998  378.076067  386.426302   
PD_0768    310.655007  313.552212  318.285458  319.786622  317.106352   
PD_0767    322.023232  326.017759  332.960584  328.612748  330.296410   
PD_0765    320.228413  316.599782  322.617785  323.439426  321.115387   
PD_0763    315.073271  324.786708  320.527911  324.333087  314.118262   
PD_0754    312.076524  301.745155  307.060159  299.297112  313.240750   
PD_0764    322.423040  316.880237  320.824348  334.203380  322.487413   
PD_0688    314.238386  312.433532  326.490682  320.355714  326.579360   
PD_0270    306.568789  314.324328  310.158640  316.150601  309.992537   
PD_0268    308.509253  316.660070  323.766100  317.850024  318.566202   
PD_0687    338.679069  334.785972  335.408568  330.479431  337.049101   
PD_0271    310.349250  309.043735  313.531271  314.698949  319.573194   
PD_0747    327.315614  328.701565  321.352745  325.549185  335.028699   
PD_0766    351.910877  346.137829  347.941321  348.325022  351.164695   
PD_0762    327.933242  333.306696  332.836010  330.718638  366.278751   
PD_0691    319.643737  328.317991  332.622380  322.074172  324.503955   
Sci_16066  350.751793  351.520294  356.059137  354.150133  360.133858   
PD_0770    296.736663  302.978707  300.366629  302.032563  302.119643   
PD_0774    292.906009  294.991814  294.706080  290.063220  303.052120   
PD_0689    326.815297  328.007562  324.176930  323.922130  323.181374   
PD_0756    317.660983  319.794005  320.576742  326.818095  326.191610   
PD_0690    307.792388  320.244172  313.258718  316.425672  312.851521   
PD_0637    326.577077  332.703332  343.265610  327.072514  325.033974   
PD_0753    310.615230  300.772318  307.361287  310.224522  305.686925   
PD_0748    308.824236  310.717362  302.346547  308.062788  303.312327   
PD_0769    315.029815  316.096039  317.583172  316.305515  314.510492   
PD_0772    314.165597  317.298603  326.488391  323.455770  325.087527   
PD_0773    297.650583  306.312838  306.404616  303.954128  311.224445   
PD_0745    318.740768  323.994572  322.662356  321.534172  321.724719   
Sci_16098  338.821162  350.016626  337.227342  332.178425  329.616997   
PD_0771    331.195249  333.276101  332.694048  328.932866  336.696224   
PD_0269    331.531153  332.563112  337.236188  340.843281  338.622859   
PD_0746    333.854126  367.592773  337.663125  342.106863  337.626411   
PD_0761    347.539525  353.870002  350.683089  356.570591  344.624177   
PD_0272    334.221025  335.899266  338.654638  342.689382  340.190522   
PD_0775    336.060834  340.188245  335.490837  333.975744  342.687565   
PD_0752    349.475137  358.508715  355.206874  358.498640  351.005724   
PD_0755    308.802330  306.882081  315.599918  314.488245  313.082025   
PD_0216    690.724991  715.961943  697.315153  698.108638  698.430690   
PD_0503    665.649166  670.539675  668.495817  680.727599  692.767635   
PD_0217    687.826893  711.823837  722.714379  720.372379  711.280045   
PD_0214    565.710624  577.150506  579.673723  569.814752  571.606558   
PD_0213    550.032000  565.233037  559.347599  564.843852  584.885083   
PD_0215    669.965817  668.619224  665.139406  654.493016  674.641589   

              PD_0721     PD_0726     PD_0722     PD_0720     PD_0743  

In [38]:
# three panels
fig, ((axA, ax1), (axB, ax2), (axC, ax3))  = plt.subplots(3, 2, sharex='col', sharey='row',
                        gridspec_kw={'height_ratios':[10,1,40], 
                                     'width_ratios':[1,40],
                                     'hspace':0.01,
                                     'wspace':0.01}, 
                        figsize=(8, 9))

# remove axes we do not need
axA.remove()
axB.remove()

# plot the tree in top panel
leaf_info = plot_tree(Tree(tree), ax1, flip=True, 
                      axis=False, linewidth=0.6,
                      markers=True, 
                      markersize=1,
                      labels=False,
                      margins=(0, 0, 0, 0), 
                      fontsize=4)

# add colored rectangles or each individual to the middle
for i, indiv in enumerate(matrix_order):
    ax2.add_patch(Rectangle((i, 0), 1, 1, linewidth=0, edgecolor='black', facecolor=species_colors[indiv]))
    #axC.add_patch(Rectangle((0, i), 1, 1, linewidth=0, edgecolor='black', facecolor=species_colors[indiv]))

# extra axes for heatmap color bar to allow x sharing of the other three axes
cbar_ax = fig.add_axes([.93, .3, .02, .3])
# plot heatmap in bottom panel
colors = ['white', 'mediumblue', 'darkviolet', 'orangered',  'darkorange'] 
cmap = LinearSegmentedColormap.from_list('custom colormap', colors, N=256)
sns.heatmap(ordered_matrix.to_numpy(), cmap=cmap, ax=ax3, cbar_ax=cbar_ax)

# add squares with labels along diagonal (maybe to label indentified clusters)
fontsize=11
for label, start, size in [('A', 50, 30), ('B', 100, 50), ('C', 150, 30)]:
    ax3.add_patch(Rectangle((start, ordered_matrix.shape[0] - start), size, - size, 
                            linewidth=0.4, edgecolor='black', facecolor='none'))
    ax3.text(start, ordered_matrix.shape[0] - start-size, label, fontsize=fontsize,
                    verticalalignment='bottom', 
                    horizontalalignment='left')    

# some additional squares (maybe to highlight particular admixture signals)
label, x, y, width, height = 'K', 20, 120, 50, 20
ax3.add_patch(Rectangle((x, ordered_matrix.shape[0] - y), width, -height, 
                        linewidth=0.4, edgecolor='black', facecolor='none'))
ax3.text(x, ordered_matrix.shape[0] - y-height, label, fontsize=fontsize,
                verticalalignment='bottom', 
                horizontalalignment='left')    
    
# remove axes from all plots
ax1.axis('off')
ax2.axis('off') 
ax3.axis('off') 
axC.axis('off') 

# lines around the matrix
ax3.axhline(y=0, color='black',linewidth=1)
ax3.axhline(y=ordered_matrix.shape[1], color='black',linewidth=1)
ax3.axvline(x=0, color='black',linewidth=1)
ax3.axvline(x=ordered_matrix.shape[0], color='black',linewidth=1)

# lines around the colorbar
cbar_ax.axhline(y=0, color='black',linewidth=1)
cbar_ax.tick_params(right=False)